In [1]:
# Predicted Visualization Based on the sample data

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

import pandas as pd
import calendar

init_notebook_mode(connected=True)

# need to replace your dataframe instead of 'df_input' here
data_path = 'data/test_data_output.csv'
df_input = pd.read_csv(data_path, sep=",")


days = df_input['DAY'].unique()
month = calendar.month_name[int(df_input['MONTH'].unique())]
# create list of predicted columns
predicted_columns = ['4440099_KWH_PRED', '4440104_KWH_PRED', '4440105_KWH_PRED', '4440177_KWH_PRED', '4440012_KWH_PRED', '4440073_KWH_PRED', 'AGGREGATE_KWH_PRED']

# create figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [0,23], 'title': 'Time(hour)', 'dtick': 1}
figure['layout']['yaxis'] = {'title': 'Predicted Meter Reading Value(kWh)'}
figure['layout']['title'] = {'text': 'Hourly Predicted Reading Value'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1',
    'plotlycommand': 'animate',
    'values': days,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 1, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 14},
        'prefix': 'Month: ' + month + '  ,  '+ 'Day: ',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
day = df_input['DAY'].unique()[0]
for predicted_column in predicted_columns:
    dataset_by_month = df_input[df_input['DAY'] == day]
    data_dict = {
        'x': list(dataset_by_month['HOUR']),
        'y': list(dataset_by_month['4440099_KWH_PRED']),
        'type': 'scatter',
        'text': predicted_columns,
        'name': predicted_column
    }
    figure['data'].append(data_dict)

# make frames
for day in days:
    frame = {'data': [], 'name': str(day)}
    for predicted_column in predicted_columns:
        dataset_by_month = df_input[df_input['DAY'] == day]

        data_dict = {
            'x': list(dataset_by_month['HOUR']),
            'y': list(dataset_by_month[predicted_column]),
            'type': 'scatter',
            'text': predicted_columns,
            'name': predicted_column
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [day],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': str(day),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)


figure['layout']['sliders'] = [sliders_dict]

iplot(figure)